# MultiMatest

## Libraries

In [1]:
import parser
import itertools
import pandas as pd

In [2]:
logic = parser.Parser(language=parser.propositional_language())
predicates = logic.language.predicates
operators = logic.language.ops
formula = "a&(a->~b)->a"
formula_parsed = logic.parse(formula)
values = list(range(2))
min_designated = 1
designated_values = list(filter(lambda x: x>=min_designated, values))

In [3]:
def unary_operator(matrix):
    return lambda x: matrix[x]

def binary_operator(matrix):
    return lambda x, y: matrix[x][y]

negation = unary_operator([1,0])

conjuntion = binary_operator([[0,0],
                              [0,1]])

disjunction = binary_operator([[0,1],
                               [1,1]])
                               
implication = binary_operator([[1,1],
                               [0,1]])

logic_functions = {'~':negation, '&':conjuntion, 'v':disjunction, '->': implication}
system = {'matrix': logic_functions, 'values': values, 'min_designated': 1}

In [4]:
inputs = formula_parsed.inputs
formula_to_interpret = list(map(lambda x: x.decode('ascii'),formula_parsed.ops.copy()))

In [5]:
formula_to_interpret

['a', 'a', 'b', '~', '->', 'a', '->', '&']

In [6]:
formula_predicates = list(sorted(set(filter(lambda x: x in logic.language.predicates, formula_to_interpret))))

In [7]:
interpretations = list(itertools.product(values, repeat = len(formula_predicates)))
replacements = [dict(zip(formula_predicates, interpretation)) for interpretation in interpretations]

In [8]:
replacements

[{'a': 0, 'b': 0}, {'a': 0, 'b': 1}, {'a': 1, 'b': 0}, {'a': 1, 'b': 1}]

In [9]:
interpret_function = list(map(logic_functions.get, formula_to_interpret, formula_to_interpret))
formula_interpretations = [list(map(replacement.get, interpret_function, interpret_function)) for replacement in replacements]

In [10]:
def evaluate_interpretation(interpretation, inputs, values):
    # The evaluation takes the values from the interpretation
    evaluation = interpretation.copy()

    # Iterates over each of the items of the interpretation
    for index, symbol in enumerate(evaluation):
        # If the symbol is a function, not a value, interpret the expression
        if not symbol in values:
            # The places where the function is applied in the interpretation are always of lower index (the parser orders the expression)
            places = list(map(lambda x: index+x, inputs[index]))
            
            # Picks the values of the places in order to feed the function
            vals = list(map(lambda x: evaluation[x], places))

            # Get the values of the function and replace the item for the value in the evaluation
            val = symbol(*vals)
            evaluation[index] = val

    return evaluation


In [11]:
def evaluate_formula_in_system(logic, system, formula, verbose = False):

    # Parse the formula
    formula_parsed = logic.parse(formula)
    inputs = formula_parsed.inputs
    formula_to_interpret = list(map(lambda x: x.decode('ascii'),formula_parsed.ops.copy()))

    # Take the logic functions for the system, the values and the minimum designated value
    logic_functions = system['matrix']
    values = system['values']
    min_designated = system['min_designated']

    # Calculate the designated values taking into account the minimum designated value
    designated_values = list(filter(lambda x: x>=min_designated, values))

    # Take the predicates to be substituted for values
    formula_predicates = list(sorted(set(filter(lambda x: x in logic.language.predicates, formula_to_interpret))))

    # Calculate all the possible interpretations and create the replacements for the predicates
    interpretations = list(itertools.product(values, repeat = len(formula_predicates)))
    replacements = [dict(zip(formula_predicates, interpretation)) for interpretation in interpretations]

    # Replace the logic symbols for their logic functions related to the matrices
    interpret_function = list(map(logic_functions.get, formula_to_interpret, formula_to_interpret))
    formula_interpretations = [list(map(replacement.get, interpret_function, interpret_function)) for replacement in replacements]

    # Evaluate the formula
    evaluations = list(map(lambda interpretation: evaluate_interpretation(interpretation, inputs, values), formula_interpretations))

    # Create DataFrame with the results
    evaluation_results = pd.DataFrame(evaluations, columns=formula_to_interpret,index=interpretations)
    evaluation_results.index.name = str(formula_predicates)

    
    if verbose:
        print(formula)
        print(evaluation_results)
        # Check if all the values of the last evaluation (last column) are in the designated values
        result_evaluation = list(map(lambda x: x in designated_values, final_results[final_results.columns[-1]].values))
        is_tautology = all(result_evaluation)

        # Select the index of the places where the formula does not hold a designated value
        fail_index = list(set([x if not y else None for x,y in enumerate(result_evaluation)]) - set([None]))
        fails_results = final_results.loc[[interpretations[index] for index in fail_index]]
        # Print if is tautological or show the places where the formula is not verified by the matrix
        if is_tautology:
            print('Tautology')
        else: 
            print(fails_results)
        
        return evaluation_results, is_tautology, fails_results
        
    else:
        
        return evaluation_results

In [23]:
formulas = ['a->a', 'a->(a&b)', 'a->(avb)', '(a->b)->((c->a)->(c->b))']

In [25]:
for formula in formulas:
    print(formula)
    print(evaluate_formula_in_system(logic, system, formula))
    print('-'*20)

a->a
       a  a  ->
['a']          
(0,)   0  0   1
(1,)   1  1   1
--------------------
a->(a&b)
            a  a  b  &  ->
['a', 'b']                
(0, 0)      0  0  0  0   1
(0, 1)      0  0  1  0   1
(1, 0)      1  1  0  0   0
(1, 1)      1  1  1  1   1
--------------------
a->(avb)
            a  a  b  v  ->
['a', 'b']                
(0, 0)      0  0  0  0   1
(0, 1)      0  0  1  1   1
(1, 0)      1  1  0  1   1
(1, 1)      1  1  1  1   1
--------------------
(a->b)->((c->a)->(c->b))
                 a  b  ->  c  a  ->  c  b  ->  ->  ->
['a', 'b', 'c']                                      
(0, 0, 0)        0  0   1  0  0   1  0  0   1   1   1
(0, 0, 1)        0  0   1  1  0   0  1  0   0   1   1
(0, 1, 0)        0  1   1  0  0   1  0  1   1   1   1
(0, 1, 1)        0  1   1  1  0   0  1  1   1   1   1
(1, 0, 0)        1  0   0  0  1   1  0  0   1   1   1
(1, 0, 1)        1  0   0  1  1   1  1  0   0   0   1
(1, 1, 0)        1  1   1  0  1   1  0  1   1   1   1
(1, 1, 1)      

In [26]:
negation = unary_operator([1,0])

conjuntion = binary_operator([[0,0],
                              [0,1]])

disjunction = binary_operator([[0,1],
                               [1,1]])
                               
implication = binary_operator([[1,1],
                               [0,1]])

classical_operators = {'~':negation, '&':conjuntion, 'v':disjunction, '->': implication}

msm4_negation = unary_operator([3,1,2,0])

msm4_conjunction = binary_operator([[0,0,0,0],
                                    [0,1,0,1],
                                    [0,0,2,2],
                                    [0,1,2,3]])

msm4_disjunction = binary_operator([[0,1,2,3],
                                    [1,1,3,3],
                                    [2,3,2,3],
                                    [3,3,3,3]])
msm4 = {'~':msm4_negation, '&':msm4_conjunction, 'v':msm4_disjunction,

        '->':binary_operator([[3,3,3,3],
                              [0,3,0,3],
                              [0,0,3,3],
                              [0,0,0,3]])
        }

msm4_1 = {'~':msm4_negation, '&':msm4_conjunction, 'v':msm4_disjunction,

        '->':binary_operator([[3,3,3,3],
                              [0,3,0,3],
                              [1,1,3,3],
                              [0,1,0,3]])
        }

msm4_2 = {'~':msm4_negation, '&':msm4_conjunction, 'v':msm4_disjunction,

        '->':binary_operator([[3,3,3,3],
                              [1,3,1,3],
                              [0,0,3,3],
                              [0,0,1,3]])
        }

systems = {
   'classical_logic': {'matrix': classical_operators, 'values': values, 'min_designated': 1},
   'MSm4': {'matrix': msm4, 'values': list(range(4)), 'min_designated': 2},
   'MSm4_1': {'matrix': msm4_1, 'values': list(range(4)), 'min_designated': 2},
   'MSm4_2': {'matrix': msm4_2, 'values': list(range(4)), 'min_designated': 2}
}

In [28]:
for system_name, system in systems.items():
    print(system_name)
    for formula in formulas:
        print(formula)
        print(evaluate_formula_in_system(logic, system, formula))
        print('-'*20)
    print('#'*20)

classical_logic
a->a
       a  a  ->
['a']          
(0,)   0  0   1
(1,)   1  1   1
--------------------
a->(a&b)
            a  a  b  &  ->
['a', 'b']                
(0, 0)      0  0  0  0   1
(0, 1)      0  0  1  0   1
(1, 0)      1  1  0  0   0
(1, 1)      1  1  1  1   1
--------------------
a->(avb)
            a  a  b  v  ->
['a', 'b']                
(0, 0)      0  0  0  0   1
(0, 1)      0  0  1  1   1
(1, 0)      1  1  0  1   1
(1, 1)      1  1  1  1   1
--------------------
(a->b)->((c->a)->(c->b))
                 a  b  ->  c  a  ->  c  b  ->  ->  ->
['a', 'b', 'c']                                      
(0, 0, 0)        0  0   1  0  0   1  0  0   1   1   1
(0, 0, 1)        0  0   1  1  0   0  1  0   0   1   1
(0, 1, 0)        0  1   1  0  0   1  0  1   1   1   1
(0, 1, 1)        0  1   1  1  0   0  1  1   1   1   1
(1, 0, 0)        1  0   0  0  1   1  0  0   1   1   1
(1, 0, 1)        1  0   0  1  1   1  1  0   0   0   1
(1, 1, 0)        1  1   1  0  1   1  0  1   1   1   1